In [1]:
import pandas as pd
import os
import glob
import csv
import numpy as np

### File aquiration template

In [2]:
# # Source file cleaning
# # Fill in the source number
# source_number = 2
# # Fill in your own file path, have the folder containing the csv files be called "Source_{number}"
# # Make sure to change any \ to / to avoid errors
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a")
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r")
#     # Split the input file in lines
#     raw_file_lines = raw_file.readlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

# # Pandas sanity check, open the just created file
# pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

### Lookup table Municipality - Province

In [3]:
# classifications of municipalities by province in the Netherlands 
df_2021 = pd.read_excel('data/Source_0/gemeenten-alfabetisch-2021.xlsx')[['Gemeentenaam', 'Provincienaam']]
df_2020 = pd.read_excel('data/Source_0/Gemeenten alfabetisch 2020.xlsx')[['Gemeentenaam', 'Provincienaam']]
# for 2018 and 2019 files, you need pip install xlrd
df_2019 = pd.read_excel('data/Source_0/Gemeenten alfabetisch 2019.xls')[['Gemeentenaam', 'Provincienaam']]
df_2018 = pd.read_excel('data/Source_0/Gemeenten alfabetisch 2018.xls')[['Gemeentenaam', 'Provincienaam']]

# 2020 and 2019 are exactly the same
# combine 2020 and 2018 in order to get all unique rows from both data sources
df_combi1820 = df_2018.merge(df_2019, on=['Gemeentenaam', 'Provincienaam'], how='outer')

# combine 2021 and the newly created 2018-2020 dataframe
df_final = df_combi1820.merge(df_2021, on=['Gemeentenaam', 'Provincienaam'], how='outer')

df_final = df_final[df_final["Provincienaam"]!="Friesland"]
df_final

,Gemeentenaam,Provincienaam
0,Aa en Hunze,Drenthe
1,Aalburg,Noord-Brabant
2,Aalsmeer,Noord-Holland
3,Aalten,Gelderland
5,Alblasserdam,Zuid-Holland
...,...,...
403,Terschelling,Fryslân
404,Tytsjerksteradiel,Fryslân
405,Vlieland,Fryslân
406,Waadhoeke,Fryslân


### Source 1 preprocessing

In [4]:
# INFO ONLY ON 2021

In [5]:
# # Source file cleaning
# source_number = 1
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a", encoding='utf-8-sig')
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r", encoding='utf-8-sig')
#     # Split the input file in lines
#     #ORIGINAL: raw_file_lines = raw_file.readlines(), leaves /r/n on last column entires
    
#     raw_file_lines = raw_file.read().splitlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

In [6]:
# Source file cleaning
source_number = 1
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_1 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

# remove "" from column names and string values
source_1.columns = source_1.columns.str[1:]
source_1.columns = source_1.columns.str[:-1]
source_1["Wijken en buurten"] = source_1["Wijken en buurten"].str[1:]
source_1["Wijken en buurten"] = source_1["Wijken en buurten"].str[:-1]
source_1["Regioaanduiding/Soort regio (omschrijving)"] = source_1["Regioaanduiding/Soort regio (omschrijving)"].str[1:]
source_1["Regioaanduiding/Soort regio (omschrijving)"] = source_1["Regioaanduiding/Soort regio (omschrijving)"].str[:-1]
# filter only municipalities
source_1 = source_1[source_1["Regioaanduiding/Soort regio (omschrijving)"]=="Gemeente  "]
# rename column for merging
source_1.rename(columns={"Wijken en buurten": "Gemeentenaam"}, inplace = True)
# drop column
source_1.drop(columns={"Regioaanduiding/Soort regio (omschrijving)"}, inplace=True)
source_1

,Gemeentenaam,Bevolking/Aantal inwoners (aantal),Bevolking/Geslacht/Mannen (aantal),Bevolking/Geslacht/Vrouwen (aantal),Bevolking/Leeftijdsgroepen/0 tot 15 jaar (aantal),Bevolking/Leeftijdsgroepen/15 tot 25 jaar (aantal),Bevolking/Leeftijdsgroepen/25 tot 45 jaar (aantal),Bevolking/Leeftijdsgroepen/45 tot 65 jaar (aantal),Bevolking/Leeftijdsgroepen/65 jaar of ouder (aantal),Bevolking/Burgerlijke staat/Ongehuwd (aantal),...,Bevolking/Geboorte en sterfte/Sterfte relatief (per 1 000 inwoners),Bevolking/Bevolkingsdichtheid (aantal inwoners per km²),Energie/Gemiddeld elektriciteitsverbruik/Gemiddeld elektriciteitsverbruik totaal (kWh),Energie/Gemiddeld aardgasverbruik/Gemiddeld aardgasverbruik totaal (m³),Opleidingsniveau/Opleidingsniveau laag (aantal),Opleidingsniveau/Opleidingsniveau middelbaar (aantal),Opleidingsniveau/Opleidingsniveau hoog (aantal),Inkomen/Inkomen van personen/Gemiddeld inkomen per inkomensontvanger (x 1 000 euro),Nabijheid voorzieningen/Afstand tot huisartsenpraktijk (km),Nabijheid voorzieningen/Afstand tot grote supermarkt (km)
0,Aa en Hunze,25399,12709,12690,3383,2611,4463,8060,6882,10191,...,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
76,Aalsmeer,31991,15913,16078,5446,3844,7069,9639,5993,15317,...,NaN,1590.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,Aalten,27120,13655,13465,4203,3011,5465,8244,6197,12154,...,NaN,281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
123,Achtkarspelen,27900,14217,13683,4757,3428,6205,7784,5726,12710,...,NaN,273.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
151,Alblasserdam,20136,9897,10239,3774,2482,4582,5268,4030,8803,...,NaN,2294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17387,Zundert,21988,11247,10741,2898,2426,4774,6919,4971,9656,...,NaN,182.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17404,Zutphen,48111,23595,24516,6930,5595,10360,14241,10985,22249,...,NaN,1175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17442,Zwartewaterland,22823,11598,11225,4626,3133,5457,5654,3953,10029,...,NaN,277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17483,Zwijndrecht,44775,21754,23021,7240,4886,10704,11774,10171,19486,...,NaN,2206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
# show nr of non-null values in each column
source_1.count()

Gemeentenaam                                                                              352
Bevolking/Aantal inwoners (aantal)                                                        352
Bevolking/Geslacht/Mannen (aantal)                                                        352
Bevolking/Geslacht/Vrouwen (aantal)                                                       352
Bevolking/Leeftijdsgroepen/0 tot 15 jaar (aantal)                                         352
Bevolking/Leeftijdsgroepen/15 tot 25 jaar (aantal)                                        352
Bevolking/Leeftijdsgroepen/25 tot 45 jaar (aantal)                                        352
Bevolking/Leeftijdsgroepen/45 tot 65 jaar (aantal)                                        352
Bevolking/Leeftijdsgroepen/65 jaar of ouder (aantal)                                      352
Bevolking/Burgerlijke staat/Ongehuwd (aantal)                                             352
Bevolking/Burgerlijke staat/Gehuwd (aantal)                 

In [8]:
# merge
merged_source_1 = pd.merge(df_final, source_1, on = "Gemeentenaam", how='right')
merged_source_1

,Gemeentenaam,Provincienaam,Bevolking/Aantal inwoners (aantal),Bevolking/Geslacht/Mannen (aantal),Bevolking/Geslacht/Vrouwen (aantal),Bevolking/Leeftijdsgroepen/0 tot 15 jaar (aantal),Bevolking/Leeftijdsgroepen/15 tot 25 jaar (aantal),Bevolking/Leeftijdsgroepen/25 tot 45 jaar (aantal),Bevolking/Leeftijdsgroepen/45 tot 65 jaar (aantal),Bevolking/Leeftijdsgroepen/65 jaar of ouder (aantal),...,Bevolking/Geboorte en sterfte/Sterfte relatief (per 1 000 inwoners),Bevolking/Bevolkingsdichtheid (aantal inwoners per km²),Energie/Gemiddeld elektriciteitsverbruik/Gemiddeld elektriciteitsverbruik totaal (kWh),Energie/Gemiddeld aardgasverbruik/Gemiddeld aardgasverbruik totaal (m³),Opleidingsniveau/Opleidingsniveau laag (aantal),Opleidingsniveau/Opleidingsniveau middelbaar (aantal),Opleidingsniveau/Opleidingsniveau hoog (aantal),Inkomen/Inkomen van personen/Gemiddeld inkomen per inkomensontvanger (x 1 000 euro),Nabijheid voorzieningen/Afstand tot huisartsenpraktijk (km),Nabijheid voorzieningen/Afstand tot grote supermarkt (km)
0,Aa en Hunze,Drenthe,25399,12709,12690,3383,2611,4463,8060,6882,...,NaN,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aalsmeer,Noord-Holland,31991,15913,16078,5446,3844,7069,9639,5993,...,NaN,1590.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Aalten,Gelderland,27120,13655,13465,4203,3011,5465,8244,6197,...,NaN,281.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Achtkarspelen,Fryslân,27900,14217,13683,4757,3428,6205,7784,5726,...,NaN,273.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Alblasserdam,Zuid-Holland,20136,9897,10239,3774,2482,4582,5268,4030,...,NaN,2294.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Zundert,Noord-Brabant,21988,11247,10741,2898,2426,4774,6919,4971,...,NaN,182.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
348,Zutphen,Gelderland,48111,23595,24516,6930,5595,10360,14241,10985,...,NaN,1175.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
349,Zwartewaterland,Overijssel,22823,11598,11225,4626,3133,5457,5654,3953,...,NaN,277.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
350,Zwijndrecht,Zuid-Holland,44775,21754,23021,7240,4886,10704,11774,10171,...,NaN,2206.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# require 30% non-null columns, more than this drops important columns
limitPer = len(merged_source_1) * 0.3
merged_source_1 = merged_source_1.dropna(thresh=limitPer, axis=1)
merged_source_1

,Gemeentenaam,Provincienaam,Bevolking/Aantal inwoners (aantal),Bevolking/Geslacht/Mannen (aantal),Bevolking/Geslacht/Vrouwen (aantal),Bevolking/Leeftijdsgroepen/0 tot 15 jaar (aantal),Bevolking/Leeftijdsgroepen/15 tot 25 jaar (aantal),Bevolking/Leeftijdsgroepen/25 tot 45 jaar (aantal),Bevolking/Leeftijdsgroepen/45 tot 65 jaar (aantal),Bevolking/Leeftijdsgroepen/65 jaar of ouder (aantal),Bevolking/Burgerlijke staat/Ongehuwd (aantal),Bevolking/Burgerlijke staat/Gehuwd (aantal),Bevolking/Burgerlijke staat/Gescheiden (aantal),Bevolking/Burgerlijke staat/Verweduwd (aantal),Bevolking/Bevolkingsdichtheid (aantal inwoners per km²)
0,Aa en Hunze,Drenthe,25399,12709,12690,3383,2611,4463,8060,6882,10191,11732,1868,1608,92.0
1,Aalsmeer,Noord-Holland,31991,15913,16078,5446,3844,7069,9639,5993,15317,13039,2135,1500,1590.0
2,Aalten,Gelderland,27120,13655,13465,4203,3011,5465,8244,6197,12154,11757,1542,1667,281.0
3,Achtkarspelen,Fryslân,27900,14217,13683,4757,3428,6205,7784,5726,12710,11739,1870,1581,273.0
4,Alblasserdam,Zuid-Holland,20136,9897,10239,3774,2482,4582,5268,4030,8803,8927,1304,1102,2294.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347,Zundert,Noord-Brabant,21988,11247,10741,2898,2426,4774,6919,4971,9656,9705,1322,1305,182.0
348,Zutphen,Gelderland,48111,23595,24516,6930,5595,10360,14241,10985,22249,18382,4995,2485,1175.0
349,Zwartewaterland,Overijssel,22823,11598,11225,4626,3133,5457,5654,3953,10029,10868,968,958,277.0
350,Zwijndrecht,Zuid-Holland,44775,21754,23021,7240,4886,10704,11774,10171,19486,18728,3834,2727,2206.0


### Source 2 preprocessing

In [10]:
# # Source file cleaning
# source_number = 2
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a", encoding='utf-8-sig')
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r", encoding='utf-8-sig')
#     # Split the input file in lines
#     #ORIGINAL: raw_file_lines = raw_file.readlines(), leaves /r/n on last column entires
    
#     raw_file_lines = raw_file.read().splitlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

In [11]:
source_number = 2
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_2 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

# remove "" from column names and string values
source_2.columns = source_2.columns.str[1:]
source_2.columns = source_2.columns.str[:-1]
source_2["Regio's"] = source_2["Regio's"].str[1:]
source_2["Regio's"] = source_2["Regio's"].str[:-1]
source_2["Perioden"] = source_2["Perioden"].str[1:]
source_2["Perioden"] = source_2["Perioden"].str[:-1]
# rename column for merging
source_2.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)

source_2

,Perioden,Gemeentenaam,Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/15 tot 20 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/20 tot 25 jaar (aantal),...,Onderwijs/Naar woongemeente/Gediplomeerden/Wo master/doctoraal (aantal),"Arbeid/Banen van werknemers, relatief/A Landbouw, bosbouw en visserij (%)","Arbeid/Banen van werknemers, relatief/B-F Nijverheid en energie (%)","Arbeid/Banen van werknemers, relatief/G-N Commerciële dienstverlening (%)","Arbeid/Banen van werknemers, relatief/O-U Niet-commerciële dienstverlening (%)",Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpraktijk (km),Nabijheid voorzieningen/Gezondheid/Aantal huisartsenpraktijken binnen 3 km (aantal),Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpost (km),Nabijheid voorzieningen/Gezondheid/Afstand tot ziekenhuis (km),Nabijheid voorzieningen/Gezondheid/Aantal ziekenhuizen binnen 20 km (aantal)
0,2019,Aa en Hunze,25386.0,12659.0,12727.0,922.0,1174.0,1366.0,1579.0,1031.0,...,8.0,1.0,10.0,45.0,44.0,"2,4","1,3","11,1","11,1","1,4"
1,2020,Aa en Hunze,25445.0,12708.0,12737.0,946.0,1160.0,1354.0,1545.0,1079.0,...,NaN,1.0,8.0,43.0,47.0,"2,4","1,1","11,1","11,1","1,4"
2,2021,Aa en Hunze,25399.0,12709.0,12690.0,937.0,1137.0,1309.0,1508.0,1103.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,Aalburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,Aalburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,2020,Zwijndrecht,44737.0,21694.0,23043.0,2348.0,2398.0,2452.0,2509.0,2408.0,...,NaN,0.0,20.0,55.0,25.0,"0,7","6,5","6,5","2,5","8,4"
2174,2021,Zwijndrecht,44775.0,21754.0,23021.0,2350.0,2370.0,2520.0,2466.0,2420.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2175,2019,Zwolle,127497.0,62794.0,64703.0,7190.0,7776.0,7708.0,7754.0,8586.0,...,122.0,0.0,10.0,48.0,42.0,1,"7,7","4,2","4,2",1
2176,2020,Zwolle,128840.0,63488.0,65352.0,7063.0,7754.0,7833.0,7785.0,8714.0,...,NaN,0.0,10.0,47.0,42.0,"0,9","7,2","4,2","4,2",1


In [12]:
# show nr of non-null values in each column
source_2.count()

Perioden                                                                                                       2178
Gemeentenaam                                                                                                   2178
Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal)                                       1062
Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal)                                        1062
Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal)                                       1062
Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal)            1062
Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal)                1062
Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal)               1062
Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen

In [13]:
merged_source_2 = pd.merge(df_final, source_2, on = "Gemeentenaam", how='right')
merged_source_2

,Gemeentenaam,Provincienaam,Perioden,Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/15 tot 20 jaar (aantal),...,Onderwijs/Naar woongemeente/Gediplomeerden/Wo master/doctoraal (aantal),"Arbeid/Banen van werknemers, relatief/A Landbouw, bosbouw en visserij (%)","Arbeid/Banen van werknemers, relatief/B-F Nijverheid en energie (%)","Arbeid/Banen van werknemers, relatief/G-N Commerciële dienstverlening (%)","Arbeid/Banen van werknemers, relatief/O-U Niet-commerciële dienstverlening (%)",Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpraktijk (km),Nabijheid voorzieningen/Gezondheid/Aantal huisartsenpraktijken binnen 3 km (aantal),Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpost (km),Nabijheid voorzieningen/Gezondheid/Afstand tot ziekenhuis (km),Nabijheid voorzieningen/Gezondheid/Aantal ziekenhuizen binnen 20 km (aantal)
0,Aa en Hunze,Drenthe,2019,25386.0,12659.0,12727.0,922.0,1174.0,1366.0,1579.0,...,8.0,1.0,10.0,45.0,44.0,"2,4","1,3","11,1","11,1","1,4"
1,Aa en Hunze,Drenthe,2020,25445.0,12708.0,12737.0,946.0,1160.0,1354.0,1545.0,...,NaN,1.0,8.0,43.0,47.0,"2,4","1,1","11,1","11,1","1,4"
2,Aa en Hunze,Drenthe,2021,25399.0,12709.0,12690.0,937.0,1137.0,1309.0,1508.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aalburg,Noord-Brabant,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aalburg,Noord-Brabant,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,Zwijndrecht,Zuid-Holland,2020,44737.0,21694.0,23043.0,2348.0,2398.0,2452.0,2509.0,...,NaN,0.0,20.0,55.0,25.0,"0,7","6,5","6,5","2,5","8,4"
2174,Zwijndrecht,Zuid-Holland,2021,44775.0,21754.0,23021.0,2350.0,2370.0,2520.0,2466.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2175,Zwolle,Overijssel,2019,127497.0,62794.0,64703.0,7190.0,7776.0,7708.0,7754.0,...,122.0,0.0,10.0,48.0,42.0,1,"7,7","4,2","4,2",1
2176,Zwolle,Overijssel,2020,128840.0,63488.0,65352.0,7063.0,7754.0,7833.0,7785.0,...,NaN,0.0,10.0,47.0,42.0,"0,9","7,2","4,2","4,2",1


In [14]:
# require 30% non-null columns, more than this drops important columns
limitPer = len(merged_source_2) * 0.3
merged_source_2 = merged_source_2.dropna(thresh=limitPer, axis=1)

In [15]:
merged_source_2

,Gemeentenaam,Provincienaam,Perioden,Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/15 tot 20 jaar (aantal),...,"Bevolking/Bevolkingsontwikkeling/Bevolkingsgroei/Bevolkingsgroei, relatief (per 1 000 inwoners)","Arbeid/Banen van werknemers, relatief/A Landbouw, bosbouw en visserij (%)","Arbeid/Banen van werknemers, relatief/B-F Nijverheid en energie (%)","Arbeid/Banen van werknemers, relatief/G-N Commerciële dienstverlening (%)","Arbeid/Banen van werknemers, relatief/O-U Niet-commerciële dienstverlening (%)",Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpraktijk (km),Nabijheid voorzieningen/Gezondheid/Aantal huisartsenpraktijken binnen 3 km (aantal),Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpost (km),Nabijheid voorzieningen/Gezondheid/Afstand tot ziekenhuis (km),Nabijheid voorzieningen/Gezondheid/Aantal ziekenhuizen binnen 20 km (aantal)
0,Aa en Hunze,Drenthe,2019,25386.0,12659.0,12727.0,922.0,1174.0,1366.0,1579.0,...,"2,3",1.0,10.0,45.0,44.0,"2,4","1,3","11,1","11,1","1,4"
1,Aa en Hunze,Drenthe,2020,25445.0,12708.0,12737.0,946.0,1160.0,1354.0,1545.0,...,"-1,8",1.0,8.0,43.0,47.0,"2,4","1,1","11,1","11,1","1,4"
2,Aa en Hunze,Drenthe,2021,25399.0,12709.0,12690.0,937.0,1137.0,1309.0,1508.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aalburg,Noord-Brabant,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aalburg,Noord-Brabant,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,Zwijndrecht,Zuid-Holland,2020,44737.0,21694.0,23043.0,2348.0,2398.0,2452.0,2509.0,...,"0,8",0.0,20.0,55.0,25.0,"0,7","6,5","6,5","2,5","8,4"
2174,Zwijndrecht,Zuid-Holland,2021,44775.0,21754.0,23021.0,2350.0,2370.0,2520.0,2466.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2175,Zwolle,Overijssel,2019,127497.0,62794.0,64703.0,7190.0,7776.0,7708.0,7754.0,...,"10,5",0.0,10.0,48.0,42.0,1,"7,7","4,2","4,2",1
2176,Zwolle,Overijssel,2020,128840.0,63488.0,65352.0,7063.0,7754.0,7833.0,7785.0,...,"7,8",0.0,10.0,47.0,42.0,"0,9","7,2","4,2","4,2",1


In [16]:
# replace , with . in strings that are supposed to be numbers
# then convert those columns to numeric

# get the non-numeric columns, except the first 3
columns_to_convert = merged_source_2.select_dtypes(exclude=[np.number]).iloc[:,3:]
# convert
for column in columns_to_convert:
    merged_source_2[column] = merged_source_2[column].str.replace(',','.')
    merged_source_2[column] = pd.to_numeric(merged_source_2[column])

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_10172/1834035879.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_source_2[column] = merged_source_2[column].str.replace(',','.')
C:\Users\t1rk9\AppData\Local\Temp/ipykernel_10172/1834035879.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_source_2[column] = pd.to_numeric(merged_source_2[column])


In [17]:
merged_source_2

,Gemeentenaam,Provincienaam,Perioden,Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/15 tot 20 jaar (aantal),...,"Bevolking/Bevolkingsontwikkeling/Bevolkingsgroei/Bevolkingsgroei, relatief (per 1 000 inwoners)","Arbeid/Banen van werknemers, relatief/A Landbouw, bosbouw en visserij (%)","Arbeid/Banen van werknemers, relatief/B-F Nijverheid en energie (%)","Arbeid/Banen van werknemers, relatief/G-N Commerciële dienstverlening (%)","Arbeid/Banen van werknemers, relatief/O-U Niet-commerciële dienstverlening (%)",Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpraktijk (km),Nabijheid voorzieningen/Gezondheid/Aantal huisartsenpraktijken binnen 3 km (aantal),Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpost (km),Nabijheid voorzieningen/Gezondheid/Afstand tot ziekenhuis (km),Nabijheid voorzieningen/Gezondheid/Aantal ziekenhuizen binnen 20 km (aantal)
0,Aa en Hunze,Drenthe,2019,25386.0,12659.0,12727.0,922.0,1174.0,1366.0,1579.0,...,2.3,1.0,10.0,45.0,44.0,2.4,1.3,11.1,11.1,1.4
1,Aa en Hunze,Drenthe,2020,25445.0,12708.0,12737.0,946.0,1160.0,1354.0,1545.0,...,-1.8,1.0,8.0,43.0,47.0,2.4,1.1,11.1,11.1,1.4
2,Aa en Hunze,Drenthe,2021,25399.0,12709.0,12690.0,937.0,1137.0,1309.0,1508.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aalburg,Noord-Brabant,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aalburg,Noord-Brabant,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,Zwijndrecht,Zuid-Holland,2020,44737.0,21694.0,23043.0,2348.0,2398.0,2452.0,2509.0,...,0.8,0.0,20.0,55.0,25.0,0.7,6.5,6.5,2.5,8.4
2174,Zwijndrecht,Zuid-Holland,2021,44775.0,21754.0,23021.0,2350.0,2370.0,2520.0,2466.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2175,Zwolle,Overijssel,2019,127497.0,62794.0,64703.0,7190.0,7776.0,7708.0,7754.0,...,10.5,0.0,10.0,48.0,42.0,1.0,7.7,4.2,4.2,1.0
2176,Zwolle,Overijssel,2020,128840.0,63488.0,65352.0,7063.0,7754.0,7833.0,7785.0,...,7.8,0.0,10.0,47.0,42.0,0.9,7.2,4.2,4.2,1.0


In [18]:
# IMPUTE MISSING VALUES:

In [19]:
# get list of all provinces
provinces = merged_source_2['Provincienaam'].unique().tolist()

# for each province
for province in provinces:
    province_data = merged_source_2[merged_source_2['Provincienaam']==province]
    # get only numeric columns from province_data
    imbuted_subset = province_data.iloc[:, 3:]
    # impute
    imbuted_subset = imbuted_subset.fillna(imbuted_subset.mean())
    # replace with imputed data
    province_data.iloc[:, 3:] = imbuted_subset
    merged_source_2[merged_source_2['Provincienaam']==province] = province_data

C:\Users\t1rk9\anaconda3\lib\site-packages\pandas\core\indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
C:\Users\t1rk9\AppData\Local\Temp/ipykernel_10172/3233709316.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_source_2[merged_source_2['Provincienaam']==province] = province_data


In [20]:
merged_source_2

,Gemeentenaam,Provincienaam,Perioden,Bevolking/Bevolkingssamenstelling op 1 januari/Totale bevolking (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Mannen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Geslacht/Vrouwen (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/Jonger dan 5 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/5 tot 10 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/10 tot 15 jaar (aantal),Bevolking/Bevolkingssamenstelling op 1 januari/Leeftijd/Leeftijdsgroepen/15 tot 20 jaar (aantal),...,"Bevolking/Bevolkingsontwikkeling/Bevolkingsgroei/Bevolkingsgroei, relatief (per 1 000 inwoners)","Arbeid/Banen van werknemers, relatief/A Landbouw, bosbouw en visserij (%)","Arbeid/Banen van werknemers, relatief/B-F Nijverheid en energie (%)","Arbeid/Banen van werknemers, relatief/G-N Commerciële dienstverlening (%)","Arbeid/Banen van werknemers, relatief/O-U Niet-commerciële dienstverlening (%)",Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpraktijk (km),Nabijheid voorzieningen/Gezondheid/Aantal huisartsenpraktijken binnen 3 km (aantal),Nabijheid voorzieningen/Gezondheid/Afstand tot huisartsenpost (km),Nabijheid voorzieningen/Gezondheid/Afstand tot ziekenhuis (km),Nabijheid voorzieningen/Gezondheid/Aantal ziekenhuizen binnen 20 km (aantal)
0,Aa en Hunze,Drenthe,2019,25386.000000,12659.000000,12727.000000,922.000000,1174.000000,1366.000000,1579.000000,...,2.300000,1.000000,10.000000,45.000000,44.000000,2.400000,1.300000,11.100000,11.100000,1.400000
1,Aa en Hunze,Drenthe,2020,25445.000000,12708.000000,12737.000000,946.000000,1160.000000,1354.000000,1545.000000,...,-1.800000,1.000000,8.000000,43.000000,47.000000,2.400000,1.100000,11.100000,11.100000,1.400000
2,Aa en Hunze,Drenthe,2021,25399.000000,12709.000000,12690.000000,937.000000,1137.000000,1309.000000,1508.000000,...,3.300000,1.583333,14.333333,44.666667,39.125000,1.741667,2.283333,8.208333,10.183333,1.433333
3,Aalburg,Noord-Brabant,2019,41522.756757,20814.183784,20708.572973,1960.410811,2083.751351,2214.783784,2466.881081,...,6.183871,3.217742,21.120968,47.008065,28.645161,1.206452,3.523387,8.374194,9.121774,3.024194
4,Aalburg,Noord-Brabant,2020,41522.756757,20814.183784,20708.572973,1960.410811,2083.751351,2214.783784,2466.881081,...,6.183871,3.217742,21.120968,47.008065,28.645161,1.206452,3.523387,8.374194,9.121774,3.024194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2173,Zwijndrecht,Zuid-Holland,2020,44737.000000,21694.000000,23043.000000,2348.000000,2398.000000,2452.000000,2509.000000,...,0.800000,0.000000,20.000000,55.000000,25.000000,0.700000,6.500000,6.500000,2.500000,8.400000
2174,Zwijndrecht,Zuid-Holland,2021,44775.000000,21754.000000,23021.000000,2350.000000,2370.000000,2520.000000,2466.000000,...,7.916000,2.585366,15.880435,52.770000,30.311111,0.906000,6.480000,6.658000,6.839000,7.120000
2175,Zwolle,Overijssel,2019,127497.000000,62794.000000,64703.000000,7190.000000,7776.000000,7708.000000,7754.000000,...,10.500000,0.000000,10.000000,48.000000,42.000000,1.000000,7.700000,4.200000,4.200000,1.000000
2176,Zwolle,Overijssel,2020,128840.000000,63488.000000,65352.000000,7063.000000,7754.000000,7833.000000,7785.000000,...,7.800000,0.000000,10.000000,47.000000,42.000000,0.900000,7.200000,4.200000,4.200000,1.000000


### Source 4 preprocessing

In [21]:
# # Source file cleaning
# source_number = 4
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a", encoding='utf-8-sig')
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r", encoding='utf-8-sig')
#     # Split the input file in lines
#     raw_file_lines = raw_file.read().splitlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

In [22]:
source_number = 4
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_4 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", on_bad_lines='skip')

# remove "" from column names and string values
source_4.columns = source_4.columns.str[1:]
source_4.columns = source_4.columns.str[:-1]
source_4["Regio's"] = source_4["Regio's"].str[1:]
source_4["Regio's"] = source_4["Regio's"].str[:-1]
source_4["Perioden"] = source_4["Perioden"].str[1:]
source_4["Perioden"] = source_4["Perioden"].str[:-1]
# rename column for merging
source_4.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)
source_4

,Status cijfer,Perioden,Gemeentenaam,Inwoners naar geslacht en leeftijd/Mannen/Totaal mannen (aantal),Inwoners naar geslacht en leeftijd/Mannen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/85 jaar of ouder (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Totaal vrouwen (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/85 jaar of ouder (aantal),Minderheden (aantal),Banen van werknemers (x 1 000),Provinciale wegen (km),Inwoners in zorginstellingen (aantal)
0,"""Definitief""",2019,Aa en Hunze,12659.0,2623.0,6912.0,1952.0,941.0,231.0,12727.0,2418.0,6820.0,1959.0,1079.0,451.0,300.0,NaN,NaN,520.0
1,"""Definitief""",2020,Aa en Hunze,12708.0,2621.0,6882.0,2001.0,965.0,239.0,12737.0,2384.0,6821.0,1971.0,1090.0,471.0,290.0,NaN,NaN,520.0
2,"""Definitief""",2021,Aa en Hunze,12709.0,2577.0,6831.0,2032.0,1016.0,253.0,12690.0,2314.0,6795.0,1992.0,1103.0,486.0,255.0,NaN,NaN,490.0
3,"""Definitief""",2019,Aalburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"""Definitief""",2020,Aalburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,"""Definitief""",2020,Buitenland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439,"""Definitief""",2021,Buitenland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1440,"""Definitief""",2019,Niet-gemeentelijk ingedeeld,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1441,"""Definitief""",2020,Niet-gemeentelijk ingedeeld,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
# drop columns with only 1 value

for col in source_4.columns:
    if len(source_4[col].unique()) == 1:
        print(col)
        source_4.drop(col,inplace=True,axis=1)
source_4

Status cijfer
Banen van werknemers (x 1 000)
Provinciale wegen (km)


,Perioden,Gemeentenaam,Inwoners naar geslacht en leeftijd/Mannen/Totaal mannen (aantal),Inwoners naar geslacht en leeftijd/Mannen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/85 jaar of ouder (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Totaal vrouwen (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/85 jaar of ouder (aantal),Minderheden (aantal),Inwoners in zorginstellingen (aantal)
0,2019,Aa en Hunze,12659.0,2623.0,6912.0,1952.0,941.0,231.0,12727.0,2418.0,6820.0,1959.0,1079.0,451.0,300.0,520.0
1,2020,Aa en Hunze,12708.0,2621.0,6882.0,2001.0,965.0,239.0,12737.0,2384.0,6821.0,1971.0,1090.0,471.0,290.0,520.0
2,2021,Aa en Hunze,12709.0,2577.0,6831.0,2032.0,1016.0,253.0,12690.0,2314.0,6795.0,1992.0,1103.0,486.0,255.0,490.0
3,2019,Aalburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020,Aalburg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,2020,Buitenland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439,2021,Buitenland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1440,2019,Niet-gemeentelijk ingedeeld,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1441,2020,Niet-gemeentelijk ingedeeld,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
# show nr of non-null values in each column
source_4.count()

Perioden                                                                  1443
Gemeentenaam                                                              1443
Inwoners naar geslacht en leeftijd/Mannen/Totaal mannen (aantal)          1062
Inwoners naar geslacht en leeftijd/Mannen/Jonger dan 20 jaar (aantal)     1062
Inwoners naar geslacht en leeftijd/Mannen/20 tot 65 jaar (aantal)         1062
Inwoners naar geslacht en leeftijd/Mannen/65 tot 75 jaar (aantal)         1062
Inwoners naar geslacht en leeftijd/Mannen/75 tot 85 jaar (aantal)         1062
Inwoners naar geslacht en leeftijd/Mannen/85 jaar of ouder (aantal)       1062
Inwoners naar geslacht en leeftijd/Vrouwen/Totaal vrouwen (aantal)        1062
Inwoners naar geslacht en leeftijd/Vrouwen/Jonger dan 20 jaar (aantal)    1062
Inwoners naar geslacht en leeftijd/Vrouwen/20 tot 65 jaar (aantal)        1062
Inwoners naar geslacht en leeftijd/Vrouwen/65 tot 75 jaar (aantal)        1062
Inwoners naar geslacht en leeftijd/Vrouwen/75 tot 85

In [25]:
merged_source_4 = pd.merge(df_final, source_4, on = "Gemeentenaam", how='right')
merged_source_4

,Gemeentenaam,Provincienaam,Perioden,Inwoners naar geslacht en leeftijd/Mannen/Totaal mannen (aantal),Inwoners naar geslacht en leeftijd/Mannen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/85 jaar of ouder (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Totaal vrouwen (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/85 jaar of ouder (aantal),Minderheden (aantal),Inwoners in zorginstellingen (aantal)
0,Aa en Hunze,Drenthe,2019,12659.0,2623.0,6912.0,1952.0,941.0,231.0,12727.0,2418.0,6820.0,1959.0,1079.0,451.0,300.0,520.0
1,Aa en Hunze,Drenthe,2020,12708.0,2621.0,6882.0,2001.0,965.0,239.0,12737.0,2384.0,6821.0,1971.0,1090.0,471.0,290.0,520.0
2,Aa en Hunze,Drenthe,2021,12709.0,2577.0,6831.0,2032.0,1016.0,253.0,12690.0,2314.0,6795.0,1992.0,1103.0,486.0,255.0,490.0
3,Aalburg,Noord-Brabant,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aalburg,Noord-Brabant,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,Buitenland,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439,Buitenland,NaN,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1440,Niet-gemeentelijk ingedeeld,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1441,Niet-gemeentelijk ingedeeld,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
# inspect rows where province is null
merged_source_4[merged_source_4["Provincienaam"].isnull()]

,Gemeentenaam,Provincienaam,Perioden,Inwoners naar geslacht en leeftijd/Mannen/Totaal mannen (aantal),Inwoners naar geslacht en leeftijd/Mannen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/85 jaar of ouder (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Totaal vrouwen (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/85 jaar of ouder (aantal),Minderheden (aantal),Inwoners in zorginstellingen (aantal)
12,Abcoude,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Abcoude,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,Abcoude,NaN,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Alkemade,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Alkemade,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1438,Buitenland,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1439,Buitenland,NaN,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1440,Niet-gemeentelijk ingedeeld,NaN,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1441,Niet-gemeentelijk ingedeeld,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
# drop rows where province is nan
merged_source_4 = merged_source_4[merged_source_4["Provincienaam"].notnull()]
merged_source_4

,Gemeentenaam,Provincienaam,Perioden,Inwoners naar geslacht en leeftijd/Mannen/Totaal mannen (aantal),Inwoners naar geslacht en leeftijd/Mannen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/85 jaar of ouder (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Totaal vrouwen (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/85 jaar of ouder (aantal),Minderheden (aantal),Inwoners in zorginstellingen (aantal)
0,Aa en Hunze,Drenthe,2019,12659.0,2623.0,6912.0,1952.0,941.0,231.0,12727.0,2418.0,6820.0,1959.0,1079.0,451.0,300.0,520.0
1,Aa en Hunze,Drenthe,2020,12708.0,2621.0,6882.0,2001.0,965.0,239.0,12737.0,2384.0,6821.0,1971.0,1090.0,471.0,290.0,520.0
2,Aa en Hunze,Drenthe,2021,12709.0,2577.0,6831.0,2032.0,1016.0,253.0,12690.0,2314.0,6795.0,1992.0,1103.0,486.0,255.0,490.0
3,Aalburg,Noord-Brabant,2019,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aalburg,Noord-Brabant,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,Zwijndrecht,Zuid-Holland,2020,21694.0,5005.0,12180.0,2519.0,1504.0,486.0,23043.0,4702.0,12693.0,2834.0,1961.0,853.0,4480.0,410.0
1433,Zwijndrecht,Zuid-Holland,2021,21754.0,5024.0,12234.0,2529.0,1506.0,461.0,23021.0,4682.0,12664.0,2863.0,1946.0,866.0,4435.0,400.0
1434,Zwolle,Overijssel,2019,62794.0,15485.0,38381.0,5466.0,2674.0,788.0,64703.0,14943.0,38843.0,5931.0,3420.0,1566.0,7300.0,1830.0
1435,Zwolle,Overijssel,2020,63488.0,15494.0,38735.0,5622.0,2798.0,839.0,65352.0,14941.0,39197.0,6108.0,3490.0,1616.0,7405.0,1860.0


In [28]:
# IMPUTE MISSING VALUES:

In [29]:
# get list of all provinces
provinces = merged_source_4['Provincienaam'].unique().tolist()

# for each province
for province in provinces:
    province_data = merged_source_4[merged_source_4['Provincienaam']==province]
    # get only numeric columns from province_data
    imbuted_subset = province_data.iloc[:, 2:]
    # impute
    imbuted_subset = imbuted_subset.fillna(imbuted_subset.mean())
    # replace with imputed data
    province_data.iloc[:, 2:] = imbuted_subset
    merged_source_4[merged_source_4['Provincienaam']==province] = province_data

C:\Users\t1rk9\anaconda3\lib\site-packages\pandas\core\indexing.py:1851: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
C:\Users\t1rk9\AppData\Local\Temp/ipykernel_10172/418457519.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_source_4[merged_source_4['Provincienaam']==province] = province_data


In [30]:
merged_source_4

,Gemeentenaam,Provincienaam,Perioden,Inwoners naar geslacht en leeftijd/Mannen/Totaal mannen (aantal),Inwoners naar geslacht en leeftijd/Mannen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Mannen/85 jaar of ouder (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Totaal vrouwen (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/Jonger dan 20 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/20 tot 65 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/65 tot 75 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/75 tot 85 jaar (aantal),Inwoners naar geslacht en leeftijd/Vrouwen/85 jaar of ouder (aantal),Minderheden (aantal),Inwoners in zorginstellingen (aantal)
0,Aa en Hunze,Drenthe,2019,12659.000000,2623.000000,6912.000000,1952.000000,941.000000,231.000000,12727.000000,2418.000000,6820.000000,1959.000000,1079.000000,451.000000,300.000000,520.00000
1,Aa en Hunze,Drenthe,2020,12708.000000,2621.000000,6882.000000,2001.000000,965.000000,239.000000,12737.000000,2384.000000,6821.000000,1971.000000,1090.000000,471.000000,290.000000,520.00000
2,Aa en Hunze,Drenthe,2021,12709.000000,2577.000000,6831.000000,2032.000000,1016.000000,253.000000,12690.000000,2314.000000,6795.000000,1992.000000,1103.000000,486.000000,255.000000,490.00000
3,Aalburg,Noord-Brabant,2019,20814.183784,4476.491892,12388.983784,2383.389189,1243.875676,321.443243,20708.572973,4249.335135,11981.475676,2414.854054,1458.340541,604.567568,2510.297297,533.72973
4,Aalburg,Noord-Brabant,2020,20814.183784,4476.491892,12388.983784,2383.389189,1243.875676,321.443243,20708.572973,4249.335135,11981.475676,2414.854054,1458.340541,604.567568,2510.297297,533.72973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1432,Zwijndrecht,Zuid-Holland,2020,21694.000000,5005.000000,12180.000000,2519.000000,1504.000000,486.000000,23043.000000,4702.000000,12693.000000,2834.000000,1961.000000,853.000000,4480.000000,410.00000
1433,Zwijndrecht,Zuid-Holland,2021,21754.000000,5024.000000,12234.000000,2529.000000,1506.000000,461.000000,23021.000000,4682.000000,12664.000000,2863.000000,1946.000000,866.000000,4435.000000,400.00000
1434,Zwolle,Overijssel,2019,62794.000000,15485.000000,38381.000000,5466.000000,2674.000000,788.000000,64703.000000,14943.000000,38843.000000,5931.000000,3420.000000,1566.000000,7300.000000,1830.00000
1435,Zwolle,Overijssel,2020,63488.000000,15494.000000,38735.000000,5622.000000,2798.000000,839.000000,65352.000000,14941.000000,39197.000000,6108.000000,3490.000000,1616.000000,7405.000000,1860.00000


### Source 5 preprocessing

In [31]:
# Currently only data on 2019, but can extend

In [32]:
# INFO ONLY ON PROVINCES

In [33]:
# # Source file cleaning
# source_number = 5
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a", encoding='utf-8-sig')
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r", encoding='utf-8-sig')
#     # Split the input file in lines
#     raw_file_lines = raw_file.read().splitlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

In [34]:
source_number = 5
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_5 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", on_bad_lines='skip')

# remove "" from column names and string values
source_5.columns = source_5.columns.str[1:]
source_5.columns = source_5.columns.str[:-1]

# remove "" and " (PV)" from column values
source_5["Regio's"] = source_5["Regio's"].str[1:]
source_5["Regio's"] = source_5["Regio's"].str[:-6]
source_5["Beroepen en specialismen"] = source_5["Beroepen en specialismen"].str[1:]
source_5["Beroepen en specialismen"] = source_5["Beroepen en specialismen"].str[:-1]
source_5["Perioden"] = source_5["Perioden"].str[1:]
source_5["Perioden"] = source_5["Perioden"].str[:-1]

# remove * char from 2020
source_5["Perioden"] = source_5["Perioden"].str.replace("*","")

# drop age, social columns
source_5.drop("Leeftijd",inplace=True,axis=1)
source_5.drop("Sociaaleconomische categorie",inplace=True,axis=1)

source_5

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_10172/243498840.py:20: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  source_5["Perioden"] = source_5["Perioden"].str.replace("*","")


,Beroepen en specialismen,Perioden,Regio's,Medisch geschoolden (aantal)
0,Totaal BIG-geregistreerde beroepen,2020,Groningen,14490
1,Totaal BIG-geregistreerde beroepen,2020,Fryslân,12270
2,Totaal BIG-geregistreerde beroepen,2020,Drenthe,10665
3,Totaal BIG-geregistreerde beroepen,2020,Overijssel,22810
4,Totaal BIG-geregistreerde beroepen,2020,Flevoland,5735
...,...,...,...,...
92,Basisarts,2019,Noord-Holland,5800
93,Basisarts,2019,Zuid-Holland,5210
94,Basisarts,2019,Zeeland,120
95,Basisarts,2019,Noord-Brabant,1905


### Source 6 preprocessing

In [35]:
# # Source file cleaning
# source_number = 6
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a", encoding='utf-8-sig')
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r", encoding='utf-8-sig')
#     # Split the input file in lines
#     raw_file_lines = raw_file.read().splitlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

In [36]:
source_number = 6
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
source_6 = pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

# remove "" from column names and string values
source_6.columns = source_6.columns.str[1:]
source_6.columns = source_6.columns.str[:-1]
source_6["Regio's"] = source_6["Regio's"].str[1:]
source_6["Regio's"] = source_6["Regio's"].str[:-1]
source_6["Perioden"] = source_6["Perioden"].str[1:]
source_6["Perioden"] = source_6["Perioden"].str[:-1]
source_6["Donorregistratie"] = source_6["Donorregistratie"].str[1:]
source_6["Donorregistratie"] = source_6["Donorregistratie"].str[:-1]

# remove * char from 2021
source_6["Perioden"] = source_6["Perioden"].str.replace("*","")

# rename column for merging
source_6.rename(columns={"Regio's": "Gemeentenaam"}, inplace = True)

source_6

C:\Users\t1rk9\AppData\Local\Temp/ipykernel_10172/624361286.py:18: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  source_6["Perioden"] = source_6["Perioden"].str.replace("*","")


,Donorregistratie,Perioden,Gemeentenaam,"Donorregister, vastgelegde keuze (x 1 000)"
0,Totaal wel of niet geregistreerd,2019,Aa en Hunze,"22,8"
1,Totaal wel of niet geregistreerd,2020,Aa en Hunze,"22,8"
2,Totaal wel of niet geregistreerd,2021,Aa en Hunze,"22,8"
3,Totaal wel of niet geregistreerd,2019,Aalsmeer,"27,3"
4,Totaal wel of niet geregistreerd,2020,Aalsmeer,"27,6"
...,...,...,...,...
3163,Niet geregistreerd in donorregister,2020,Zwijndrecht,"20,9"
3164,Niet geregistreerd in donorregister,2021,Zwijndrecht,"3,2"
3165,Niet geregistreerd in donorregister,2019,Zwolle,"59,1"
3166,Niet geregistreerd in donorregister,2020,Zwolle,"56,2"


In [37]:
source_6.count()

Donorregistratie                              3168
Perioden                                      3168
Gemeentenaam                                  3168
Donorregister, vastgelegde keuze (x 1 000)    3168
dtype: int64

In [38]:
# merge
merged_source_6 = pd.merge(df_final, source_6, on = "Gemeentenaam", how='right')
merged_source_6

,Gemeentenaam,Provincienaam,Donorregistratie,Perioden,"Donorregister, vastgelegde keuze (x 1 000)"
0,Aa en Hunze,Drenthe,Totaal wel of niet geregistreerd,2019,"22,8"
1,Aa en Hunze,Drenthe,Totaal wel of niet geregistreerd,2020,"22,8"
2,Aa en Hunze,Drenthe,Totaal wel of niet geregistreerd,2021,"22,8"
3,Aalsmeer,Noord-Holland,Totaal wel of niet geregistreerd,2019,"27,3"
4,Aalsmeer,Noord-Holland,Totaal wel of niet geregistreerd,2020,"27,6"
...,...,...,...,...,...
3163,Zwijndrecht,Zuid-Holland,Niet geregistreerd in donorregister,2020,"20,9"
3164,Zwijndrecht,Zuid-Holland,Niet geregistreerd in donorregister,2021,"3,2"
3165,Zwolle,Overijssel,Niet geregistreerd in donorregister,2019,"59,1"
3166,Zwolle,Overijssel,Niet geregistreerd in donorregister,2020,"56,2"


In [39]:
# replace , with . in strings that are supposed to be numbers
# then convert those columns to numeric

# get the non-numeric columns, except the first 3
columns_to_convert = merged_source_6.select_dtypes(exclude=[np.number]).iloc[:,3:]
# convert
for column in columns_to_convert:
    merged_source_6[column] = merged_source_6[column].str.replace(',','.')
    merged_source_6[column] = pd.to_numeric(merged_source_6[column])
    
merged_source_6

,Gemeentenaam,Provincienaam,Donorregistratie,Perioden,"Donorregister, vastgelegde keuze (x 1 000)"
0,Aa en Hunze,Drenthe,Totaal wel of niet geregistreerd,2019,22.8
1,Aa en Hunze,Drenthe,Totaal wel of niet geregistreerd,2020,22.8
2,Aa en Hunze,Drenthe,Totaal wel of niet geregistreerd,2021,22.8
3,Aalsmeer,Noord-Holland,Totaal wel of niet geregistreerd,2019,27.3
4,Aalsmeer,Noord-Holland,Totaal wel of niet geregistreerd,2020,27.6
...,...,...,...,...,...
3163,Zwijndrecht,Zuid-Holland,Niet geregistreerd in donorregister,2020,20.9
3164,Zwijndrecht,Zuid-Holland,Niet geregistreerd in donorregister,2021,3.2
3165,Zwolle,Overijssel,Niet geregistreerd in donorregister,2019,59.1
3166,Zwolle,Overijssel,Niet geregistreerd in donorregister,2020,56.2


### Source 7 preprocessing

In [40]:
# # Source file cleaning
# source_number = 7
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a", encoding='utf-8-sig')
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r", encoding='utf-8-sig')
#     # Split the input file in lines
#     raw_file_lines = raw_file.read().splitlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

In [41]:
source_number = 7
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"""Geslacht""","""Leeftijd""","""Geneesmiddelengroep (ATC)""","""Perioden""","""Regio's""","""Personen met geneesmiddelen, relatief (%)"""
0,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2019""","""Aa en Hunze""","44,33"
1,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2020*""","""Aa en Hunze""",39
2,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2019""","""Aalburg""",NaN
3,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2020*""","""Aalburg""",NaN
4,"""Mannen""","""0 tot 15 jaar""","""Totaal""","""2019""","""Aalsmeer""","48,55"
...,...,...,...,...,...,...
13659,"""Vrouwen""","""75 jaar of ouder""","""Totaal""","""2020*""","""Zwartewaterland""","88,11"
13660,"""Vrouwen""","""75 jaar of ouder""","""Totaal""","""2019""","""Zwijndrecht""","89,41"
13661,"""Vrouwen""","""75 jaar of ouder""","""Totaal""","""2020*""","""Zwijndrecht""","89,64"
13662,"""Vrouwen""","""75 jaar of ouder""","""Totaal""","""2019""","""Zwolle""","89,72"


### Source 9 preprocessing

In [42]:
# # Source file cleaning
# source_number = 9
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a", encoding='utf-8-sig')
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r", encoding='utf-8-sig')
#     # Split the input file in lines
#     raw_file_lines = raw_file.read().splitlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

In [43]:
source_number = 9
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"""Geslacht""","""Leeftijd""","""Soort opname""","""Diagnose""","""Perioden""","""Regio's""","""Opnamen per 10 000 inwoners (per 10 000 inwoners)"""
0,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Aa en Hunze""","6874,8"
1,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Aalburg""",NaN
2,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Aalsmeer""","7042,1"
3,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Aalten""","7558,6"
4,"""Mannen""","""0 jaar""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Achtkarspelen""","9019,8"
...,...,...,...,...,...,...,...
5119,"""Vrouwen""","""80 jaar of ouder""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Zundert""","3212,7"
5120,"""Vrouwen""","""80 jaar of ouder""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Zutphen""","3625,6"
5121,"""Vrouwen""","""80 jaar of ouder""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Zwartewaterland""","3108,6"
5122,"""Vrouwen""","""80 jaar of ouder""","""Totaal opnamen""","""Totaal alle diagnosen""","""2019*""","""Zwijndrecht""","3811,3"


### Source 11 preprocessing

In [44]:
# # Source file cleaning
# source_number = 11
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a", encoding='utf-8-sig')
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r", encoding='utf-8-sig')
#     # Split the input file in lines
#     raw_file_lines = raw_file.read().splitlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

In [45]:
source_number = 11
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"""Perioden""","""Regio's""","""Totaal alle onderliggende doodsoorzaken (aantal)""","""Nieuwvormingen (aantal)""","""Ziekten van hart en vaatstelsel (aantal)""","""Ziekten van ademhalingsstelsel (aantal)"""
0,"""2018""","""Aa en Hunze""",283.0,96.0,79.0,14.0
1,"""2019""","""Aa en Hunze""",280.0,94.0,86.0,18.0
2,"""2020""","""Aa en Hunze""",274.0,88.0,77.0,14.0
3,"""2018""","""Aalburg""",84.0,27.0,23.0,8.0
4,"""2019""","""Aalburg""",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2131,"""2019""","""Zwijndrecht""",471.0,154.0,121.0,32.0
2132,"""2020""","""Zwijndrecht""",558.0,142.0,123.0,32.0
2133,"""2018""","""Zwolle""",988.0,287.0,266.0,74.0
2134,"""2019""","""Zwolle""",953.0,294.0,258.0,65.0


### Source 14 preprocessing

In [46]:
# # Source file cleaning
# source_number = 14
# source_folder_path = f"data/Source_{source_number}"

# # Open the source folder to take all CSV's from
# csv_files_Source = glob.glob(os.path.join(f"{source_folder_path}", "*.csv"))

# # Open the file to store the new CSV in
# compiled_file_source = open(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv", "a", encoding='utf-8-sig')
# # Start the CSV writer
# compiled_file_source_writer = csv.writer(compiled_file_source)

# # Go through all raw source files in the provided folder
# for file_path in csv_files_Source:
#     raw_file = open(file_path, "r", encoding='utf-8-sig')
#     # Split the input file in lines
#     raw_file_lines = raw_file.read().splitlines()
#     for line in raw_file_lines:
#         # Split each line on the column split symbol ";"
#         raw_line_split = line.split(';')
#         # Add the split row to the new file
#         compiled_file_source_writer.writerow(raw_line_split)
#     raw_file.close()
# compiled_file_source.close()

In [47]:
source_number = 14
source_folder_path = f"data/Source_{source_number}"

# Pandas sanity check, open the just created file
pd.read_csv(f"{source_folder_path}/Source_{source_number}_full_converted_raw.csv")

,"""Perioden""","""Regio's""","""Bevolking aan het begin van de periode (aantal)""","""Levend geboren kinderen (aantal)""","""Overledenen (aantal)""","""Vertrek uit de gemeente/Vertrek naar andere gemeente (aantal)""","""Bevolkingsgroei/Bevolkingsgroei, relatief (%)"""
0,"""2019""","""Aa en Hunze""",25386.0,176.0,280.0,1305.0,"0,23"
1,"""2020""","""Aa en Hunze""",25445.0,167.0,274.0,1395.0,"-0,18"
2,"""2021*""","""Aa en Hunze""",25399.0,198.0,284.0,1182.0,"0,69"
3,"""2019""","""Aalburg""",NaN,NaN,NaN,NaN,NaN
4,"""2020""","""Aalburg""",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1660,"""2020""","""Zwijndrecht""",44737.0,406.0,558.0,1969.0,"0,08"
1661,"""2021*""","""Zwijndrecht""",44775.0,490.0,535.0,2289.0,"-0,01"
1662,"""2019""","""Zwolle""",127497.0,1460.0,953.0,5356.0,"1,05"
1663,"""2020""","""Zwolle""",128840.0,1469.0,1036.0,5574.0,"0,78"
